<a href="https://colab.research.google.com/github/robfalck/dymos_tutorial/blob/main/00_dymos_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dymos: The Basics

Dymos is a library for optimal control built on top of [OpenMDAO](https://github.com/OpenMDAO/OpenMDAO).
There are several quality optimal control packages available out there, both commercial and free, so why use Dymos?

1. OpenMDAO is a framework for multidisciplinary optimization.  It allows derivatives for gradient-based optimization to be calculated across complex models, even models which include iterative solvers.

2. Dymos makes use of gradient-based optimization of nonlinear transcriptions of optimal control problems: primarily Gauss-Lobatto collocation and the Radau Pseudospectral Method.  Using these techniques relies on the calculation of accurate derivatives across the model:  _If the initial state or parameter affecting a system is changed by x, how much does the final state change?_

# Installing Dymos 

In [1]:
!pip install dymos

# Simulating the flight of a basic projectile

Dymos can simulate the dynamics of systems which are defined by systems of **ordinary differential equations** or (in some cases) **differential algebraic equations**.

As a "Hello, World!" for Dymos, consider the differential equations which govern the flight of a projectile in a 2D, rectilinear, constant gravity field:

\begin{align}
  \ddot{x} &= 0 \\
  \ddot{y} &= -g
\end{align}

Here $x$ is the horizontal position of the projectile, $y$ is the height, and $g$ is the acceleration due to gravity.

To simulate this system in Dymos, we first convert this system of 2nd-order differential equations into 1st-order differential equations:

\begin{align}
  \dot{x} &= v_x \\
  \dot{y} &= v_y \\
  \dot{v_x} &= 0 \\
  \dot{v_y} &= -g
\end{align}

# The ODE as an OpenMDAO System

Dymos expects the ODE to be provided as an OpenMDAO _System_ - either a single Component, or a Group of components for more complex systems.

To define our system, we'll first need to import some packages.
Since we're not using parallel processing for this demo, we can ignore the warnings about `mpi4py` and `petsc4py`.

In [2]:
import numpy as np
import openmdao.api as om
import dymos as dm

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


The ODE System itself in this case is an OpenMDAO ExplicitComponent.  In most cases, ExplicitComponents used as ODE's in Dymos will need the following three methods defined:

## `initialize`

This method is used to declare options for Components and Groups in OpenMDAO.  In OpenMDAO, any option we create may be used as an initialization argument when we instantiate the class.  Dymos will always pass an option `num_nodes` to the ODE class.  This is the number of points at which the ODE is simultaneously evaluated.  This option is mandatory (it has no default value).

## `setup`

This is where inputs and outputs are added to our system.  In our case, the ODE accepts two inputs ('vx' and 'vy') and computes the time derivatives of our states: `x`, `y`, `vx`, and `vy`.

For this system, the derivatives of the states `x` and `y` are provided by the states `vx` and `vy`.  Therefore it isn't necessary to have our ODE compute their rates, but in this case we will do so for simplicity.

We use the OpenMDAO notion of `tags` to let Dymos know that a given output should be used to provide the rate of a state variable.  This makes the ODE a bit more modular, and prevents us from having to explicitly tell Dymos where to find the state's rate every time we use this ODE.  The two relevant tags here are:

- state_rate_source:foo

Provides the state rate source for the state named `foo`.

- state_units:str

Specify the default units for the associated state.

Finally, we use the `declare_partials` method here to tell OpenMDAO that we want to approximate the partials of each output with respect to each input as a dense matrix using finite-difference differentiation.  Later, we'll demonstrate how to improve the efficiency of Dymos by being smarter about how we compute our derivatives.

## `compute`

The compute method is where we take values from our inputs then compute and populate the outputs.  The `inputs` and `outputs` are both provided as a dictionary-like interface.

In [3]:
class ProjectileODE(om.ExplicitComponent):

  def initialize(self):
    self.options.declare('num_nodes', types=int,
                         desc='the number of points at which this ODE is simultaneously evaluated')

  def setup(self):
    nn = self.options['num_nodes']
    self.add_input('vx', shape=(nn,), units='m/s')
    self.add_input('vy', shape=(nn,), units='m/s')

    self.add_output('x_dot', shape=(nn,), units='m/s',
                    tags=['state_rate_source:x', 'state_units:m'])

    self.add_output('y_dot', shape=(nn,), units='m/s',
                    tags=['state_rate_source:y', 'state_units:m'])

    self.add_output('vx_dot', shape=(nn,), units='m/s**2',
                    tags=['state_rate_source:vx', 'state_units:m/s'])

    self.add_output('vy_dot', shape=(nn,), units='m/s**2',
                    tags=['state_rate_source:vy', 'state_units:m/s'])
    
    self.declare_partials(of='*', wrt='*', method='fd')
    
  def compute(self, inputs, outputs):
    outputs['x_dot'][:] = inputs['vx']
    outputs['y_dot'][:] = inputs['vy']
    outputs['vx_dot'][:] = 0.0
    outputs['vy_dot'][:] = -9.81
    


# Using the ODE

Let's simulate the trajectory of a cannonball (assuming no drag) using our ODE.  There are a few ways to do this in Dymos:

1. Explicitly - We can let `scipy.integrate.solve_ivp` propagate the ODE from some initial state.

2. Implicitly - We can guess what the trajectory of the cannonball looks like over some set of discrete points in time and then use a nonlinear solver or an optimizer to iterate on that initial guess until it matches the dynamics given by the ODE.

When solving large optimal control problems, Dymos will use the implicit approach to find control profile which minimizes some objective.

The explicit simulation is typically used to generate an initial guess or to check the accuracy of the implicit solution.


# Phases

The fundamental building block of trajectories in Dymos is the _Phase_.  A phase represents some portion of the trajectory in time.  We can break complex trajectories into many phases and then link them together to provide continuity in the states, but in a simple system we can often use a single phase to represent the entire trajectory.



# Transcriptions

Phases can be seen as a data structure which combines our ODE with some _transcription_.  In Dymos, a _transcription_ is a way of converting the _continous_ problem (finding polynomial representations of the optimal state and control histories of a system) into a _discrete_ representation that can be solved by a nonlinear solver or an nonlinear programming optimization algorithm.

The transcriptions in Dymos break a Phase into one or more segments.  On each segment, each state and each control is represented as a polynomial in non-dimensional time.  Systems in which the states are highly nonlinear, with lots of oscillations, generally require a lot of segments.  Our cannonball system here is pretty benign, and can be accurately modeled by a single cubic polynomial segment (the lowest order possible in Dymos).


# Creating the OpenMDAO Problem

So without further ado, let's instantiate an OpenMDAO problem and model our system as a Trajectory with a single Phase.

In [4]:
prob = om.Problem()

traj = prob.model.add_subsystem('traj', dm.Trajectory())

phase = traj.add_phase('phase', dm.Phase(ode_class=ProjectileODE, transcription=dm.Radau(num_segments=1, order=3)))

# Propagating the ODE from some initial condition for some set amount of time

In Dymos, each Phase has some _special_ variables: time, states, controls, and parameters.  To propagate our ODE, we'll need to apply some rules to our time and states - this problem has no control variables or parameters.

First, we'll tell Dymos that the initial time for the Phase should not be treated as a design variable for the optimization.  Whatever initial time we specify will remain unchanged throughout the optimization.

The duration of the phase is, in this case, limited to positive values from 1 second to 1000 seconds.  When optimizing the system, these _bounds_ prevent the optimizer from trying nonsensical values (negative times or a duratio of zero seconds.

In [5]:
phase.set_time_options(fix_initial=True, duration_bounds=(1, 1000), units='s')

Similarly, we're going to propagate the system from some known initial state, so we'll specify that that value is fixed.

In [6]:
phase.set_state_options('x', fix_initial=True)
phase.set_state_options('y', fix_initial=True)
phase.set_state_options('vx', fix_initial=True)
phase.set_state_options('vy', fix_initial=True)

Next we call OpenMDAO's `Problem.setup` method.  This works out the various connections between the different systems that Dymos uses to solve this problem and allocates memory.  You can think of this step a bit like compiling our model before we run it.

In [7]:
prob.setup()

Now that the problem has been setup, we can specify guesses for the variables in our problem.

`<path_to_phase>.t_initial` - The initial time of the given phase.  
`<path_to_phase>.t_duration` - The time duration of the given phase.

In [8]:
prob.set_val('traj.phase.t_initial', 0.0, units='s')
prob.set_val('traj.phase.t_duration', 15.0, units='s')

We provide guesses for the states across the phase.  Since it's difficult to know where the nodes of the transcription fall in time, Dymos provides some useful interpolation methods that can be used to provide guesses to the states in time.

Since `fix_initial` is `True`, the given value at the first node will not be changed throughout an optimization.

In [9]:
prob.set_val('traj.phase.states:x', phase.interpolate(ys=[0, 100], nodes='state_input'), units='m')
prob.set_val('traj.phase.states:y', phase.interpolate(ys=[0, 0], nodes='state_input'), units='m')
prob.set_val('traj.phase.states:vx', phase.interpolate(ys=[100, 100], nodes='state_input'), units='m/s')
prob.set_val('traj.phase.states:vy', phase.interpolate(ys=[100, -100], nodes='state_input'), units='m/s')

# Propagating the Trajectory

We'll use Dymos' convenience method, `run_problem` to execute the model without optimization (`run_driver=False`) and then simulate the problem from the given initial states for the given duration (`simulate=True`).

Results of the optimization are automatically saved to the OpenMDAO record file `dymos_solution.db`, while the simulation is saved to `dymos_simulation.db`.  Plots of all of our timeseries variables (states and state rates in this case) are made and saved in the `plots` subdirectory.

In [13]:
dm.run_problem(prob, run_driver=False, simulate=True, make_plots=True)


Simulating trajectory traj
Done simulating trajectory traj


In [15]:
# TODO: Make these more automatic (and use an interactive plot library like bokeh)

from IPython.display import display
from ipywidgets import widgets, GridBox

items = [widgets.Image(value=open(f'plots/states_{state}.png', 'rb').read()) for state in ['x', 'y', 'vx', 'vy']] + \
[widgets.Image(value=open(f'plots/state_rates_{state}.png', 'rb').read()) for state in ['x', 'y', 'vx', 'vy']]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2, 500px)"))


GridBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01 \x08\x06\x00\x…

The plots above show the _implicit_ solution of the states over time (as discrete points) in comparison to a more continuous integration of the states generated by `scipy.integrate.solve_ivp`.

There is disagreement between these two solutions.  Since no iteration was performed, the discrete points represent the interpolated initial guess we specified above, while the simulation results are more accurate (to the precision provided by the methods underlying `scipy.integrate.solve_ivp`.)

# Summary

The simulate method in Dymos is useful to see how the dynamics evolve, but does not currently provide a useful way of targeting or optimizing a trajectory.

For that, we need to use a nonlinear solver or optimizer to satisfy constraints in the system.

# Boundary Value Problem:  Vary duration such that the cannonball hits the ground

In order to solve a boundary value problem, we have to allow the system to iterate.  With OpenMDAO, that gives us two options:

1. Use an optimization driver to enforce the constraints.
2. Use a nonlinear solver to enforce the constraints.